# Capstone Course
## Week 03 Assignment
### Part 2 - Data scrapping of Canada postal codes
### Adding Longitud and latitud coordinates


### Relevant note:
Part 1 of the data scrape is repeated

Part 2 that inlcudes code to add longitud and latitud, starts in cell number 30

In [3]:
# Inlcude Beutifulsoup library
from bs4 import BeautifulSoup


In [4]:
# Open the file directly from its URL
import urllib3
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
http = urllib3.PoolManager()
page = http.request('GET', url)
bs = BeautifulSoup(page.data)

# print(bs.prettify()) #to verify that we get the html file correctly


D:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [5]:
# Obtain the table information 

table = bs.find(lambda tag: tag.name=='tbody')  


In [6]:
# Process the information we have from the html table
# Since the format changed from a regular 3 column table to a grid,
# and will process each 'td' table data cell, and clean the information
# while we iterate between the cells

import pandas as pd

# Identify the total amount of cells, that will be the same as
# the total rows of our data frame
count_rows = 0
for row in table.find_all('tr'):
    for column in row.find_all('td'):
        count_rows +=1

# Create a dataframe so we can fill it with the scraped data

canada_codes = pd.DataFrame(columns = ['PostalCode','Borough','Neighbourhood'], index = range(0,count_rows))

# Process the cells in modified wiki table

cell_marker = 0
for row in table.find_all('tr'):
    for column in row.find_all('td'):
        text_string = column.get_text().strip('\n')
        postal_code = text_string[0:3]
        # Use the first parenthesis as a basis to separate bourghs from neighbourhoods
        first_parenthesis = text_string.find('(',3)
        if first_parenthesis == -1:
            # Catch an expection to the '(' rule, where its not consisten and a '/' is used
            if text_string.find('/',3) == -1:
                borough = text_string[3: len(text_string)]
                neighbourhood = 'NaN'
            else:
                borough = text_string[3: text_string.find('/',3)]
                neighbourhood = text_string[text_string.find('/',3):len(text_string)].replace('/','')
        else:
            borough = text_string[3: first_parenthesis]
            #And replace and clean the neighberhood string data
            neighbourhood = text_string[first_parenthesis : len(text_string)].replace('/',', ').replace('(','').replace(')','')
        
        # Insert the cell data into our dataframe
        canada_codes.iat[cell_marker,0] = postal_code
        canada_codes.iat[cell_marker,1] = borough
        canada_codes.iat[cell_marker,2] = neighbourhood
        
        cell_marker += 1

# Review the inserted data
canada_codes.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


In [7]:
# Create a copy to manipulate to transform the canada_codes table, eliminating unecesary boroughs 'Not assigned'
df_copy = canada_codes[canada_codes.Borough != 'Not assigned'].copy()

df_copy.head(5)


,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


In [8]:
# We confirm there are no Neighberhoods with "Not assigned"
df_copy_2 = df_copy[df_copy.Neighbourhood == 'Not assigned'].copy()
df_copy_2.head(5)


,PostalCode,Borough,Neighbourhood


In [9]:
# We review the shape of the datafram, resulting in 103 rows with 3 columns
df_copy.shape

(103, 3)

### Week 03 Part 1 of Assignment - Ends
### Week 03 Part 2 of Assignment - Begins

Tested geocoder library, but confirmed its unreliable.

### So the use of the CSV file for the coordinates was used.


In [2]:
# import geocoder
# import geocoder 

# initialize your variable to None
# lat_lng_coords = None
# postal_code = 'M5G'
# loop until you get the coordinates
# while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

In [11]:
# We import the latitude and longitude data

df_lat_lng = pd.read_csv ('Geospatial_Coordinates.csv')

df_lat_lng.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
# And combine information with the initial dataframe

merge_df = pd.merge(left=df_copy, right=df_lat_lng, how='left', left_on='PostalCode', right_on='Postal Code')

merge_df.head(5)

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


In [30]:
# And we remove unecesary columns to match the required data frame

merge_df.drop(['Postal Code'], axis = 1, inplace = True)
merge_df.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don MillsNorth,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [31]:
merge_df.shape

(103, 5)

### Merging the data results in a 103 row with 5 column data frame
